In [1]:
import pandas as pd
import json
import numpy as np
import math

In [2]:
import functools as ft

> Link muscle group in exrx and gym building exercises

> Link those with the anatomy of yoga

> seperate anatomy and muscles

>

# Exract Yoga Data

### Pose Details

In [3]:
pose_details_dict = {
                 'name':[], 'description':[], 'target_areas':[],'benefits': [],
                 'preparatory_poses':[], 'followup_poses':[],'instructions':[],'variation':[]}

with open('..\Data_Scrape_Yoga\yoga\Extracted\pose_details.jsonl', 'r', encoding="utf8") as json_file:
    json_list = list(json_file)
    
for json_str in json_list:
    result = json.loads(json_str)
    
    pose_details_dict['name'].append(result['name'])
    pose_details_dict['description'].append(result['description'])
    pose_details_dict['target_areas'].append(result['target_areas'])
    pose_details_dict['benefits'].append(result['benefits'])
    pose_details_dict['preparatory_poses'].append(result['preparatory_poses'])
    pose_details_dict['followup_poses'].append(result['followup_poses'])
    pose_details_dict['instructions'].append(result['intructions'])
    pose_details_dict['variation'].append(result['variation'])
    
yoga_pose_df = pd.DataFrame.from_dict(pose_details_dict)    

In [4]:
yoga_pose_df.head()

,name,description,target_areas,benefits,preparatory_poses,followup_poses,instructions,variation
0,Child’s Pose,[Child’s Pose (Balasana) centers on creating a...,[Full Body],[Other Child’s Pose perks:],[Balasana can appear at the beginning of class...,[],[Kneel on the floor. Touch your big toes toget...,"[Child’s Pose with a bolster, Begin on your ha..."
1,Bow Pose,[Bend back into the shape of a bow to feel ene...,[Core],[Avoid or modify this pose if you have high or...,"[Before attempting this intense pose, bring yo...","[Balasana (Child’s Pose), Pavanamuktasana (Win...",[Begin on your belly with your legs hip-distan...,"[Photo: Andrew Clark; Clothing: Calia, Bow Pos..."
2,Camel Pose,[Camel Pose (Ustrasana) is an energizing and b...,[Core],[Additional Camel Pose perks:],[Bhujangasana (Cobra Pose)],"[Paschimottanasana (Seated Forward Bend), Anan...","[Come to your knees, with your legs hip-width ...","[Photo: Andrew Clark; Clothing: Calia, Camel P..."
3,Chair Pose,[Utkatasana (Chair Pose) is sometimes translat...,[Lower body],[The secret to a stable Utkatasana is releasin...,[Tadasana (Mountain Pose)],"[Uttanasana (Standing Forward Bend), Setu Band...","[Stand in, Tadasana, . Inhale and raise your a...","[Photo: Andrew Clark, Chair Pose with a block,..."
4,Cat Pose,[You will encounter Marjaryasana (Cat Pose) an...,[Core],[Other Cat Pose perks],[Tabletop],[Bitilasana (Cow Pose)],[Begin Tabletop with your hips directly over y...,"[Photo: Andrew Clark, Cat Pose with blocks, If..."


In [5]:
yoga_pose_df.shape

(92, 8)

In [6]:
yoga_pose_df.isnull().sum()

name                 0
description          0
target_areas         0
benefits             0
preparatory_poses    0
followup_poses       0
instructions         0
variation            0
dtype: int64

### Pose Table

In [7]:
pose_table_dict = {
                 'english_Name':[], 'sanskrit_Name':[], 'pose_Type':[],'url': []}

with open('..\Data_Scrape_Yoga\yoga\Extracted\poseTable.jsonl', 'r', encoding="utf8") as json_file:
    json_list = list(json_file)
    
for json_str in json_list:
    result = json.loads(json_str)
    
    pose_table_dict['english_Name'].append(result['english_Name'])
    pose_table_dict['sanskrit_Name'].append(result['sanskrit_Name'])
    pose_table_dict['pose_Type'].append(result['pose_Type'])
    pose_table_dict['url'].append(result['url'])

yoga_details_df = pd.DataFrame.from_dict(pose_table_dict)    

In [8]:
yoga_details_df.head()

,english_Name,sanskrit_Name,pose_Type,url
0,Bharadvaja's Twist,Bharadvajasana I,"[Hip-Opening Yoga Poses, Seated Yoga Poses, Tw...",https://www.yogajournal.com/poses/bharadvaja-s...
1,Big Toe Pose,Padangusthasana,"[Forward Bend Yoga Poses, Standing Yoga Poses]",https://www.yogajournal.com/poses/big-toe-pose/
2,Boat Pose,Paripurna Navasana,"[Core Yoga Poses, Seated Yoga Poses, Strengthe...",https://www.yogajournal.com/poses/full-boat-po...
3,Bound Angle Pose,Baddha Konasana,"[Forward Bend Yoga Poses, Hip-Opening Yoga Pos...",https://www.yogajournal.com/poses/bound-angle-...
4,Bow Pose,Dhanurasana,"[Chest-Opening Yoga Poses, Yoga Backbends]",https://www.yogajournal.com/poses/bow-pose/


In [9]:
yoga_details_df.shape

(92, 4)

### Yoga categories

In [10]:
pose_category_dict = {
                 'anatomy':[], 'url':[]}

with open('..\Data_Scrape_Yoga\yoga\Extracted\yoga_categories.jsonl', 'r', encoding="utf8") as json_file:
    json_list = list(json_file)
    
for json_str in json_list:
    result = json.loads(json_str)
    pose_category_dict['anatomy'].append(result['anatomy'])
    pose_category_dict['url'].append(result['url'])

yoga_categories_df = pd.DataFrame.from_dict(pose_category_dict)    

In [11]:
yoga_categories_df.head()

,anatomy,url
0,Abs,https://www.yogajournal.com/poses/anatomy/abs/
1,Ankles,https://www.yogajournal.com/poses/anatomy/ankles/
2,Arms,https://www.yogajournal.com/poses/anatomy/arms/
3,Back,https://www.yogajournal.com/poses/anatomy/back/
4,Belly,https://www.yogajournal.com/poses/anatomy/belly/


In [12]:
yoga_categories_df.shape

(24, 2)

In [13]:
yoga_categories_df.anatomy.unique()

array(['Abs', 'Ankles', 'Arms', 'Back', 'Belly', 'Glutes', 'Hamstrings',
       'Hands', 'Flexors', 'Hips', 'Heart', 'Kidneys', 'Knees', 'Legs',
       'Lungs', 'Neck', 'Pelvis', 'Sacrum', 'Shoulders', 'Spine',
       'Thighs', 'Wrists'], dtype=object)

### Yoga poses

In [14]:
poses_dict = {
                 'anatomy':[], 'pose_name':[],'url': []}

with open('..\Data_Scrape_Yoga\yoga\Extracted\yoga_poses.jsonl', 'r', encoding="utf8") as json_file:
    json_list = list(json_file)
    
for json_str in json_list:
    result = json.loads(json_str)
    
    poses_dict['anatomy'].append(result['anatomy'])
    poses_dict['pose_name'].append(result['pose_name'])
    poses_dict['url'].append(result['url'])

yoga_pose_df = pd.DataFrame.from_dict(poses_dict)    

In [15]:
yoga_pose_df.head()

,anatomy,pose_name,url
0,Wrists,Crow Pose | Crane Pose,https://www.yogajournal.com/poses/crane-pose/
1,Wrists,Eight-Angle Pose,https://www.yogajournal.com/poses/eight-angle-...
2,Wrists,Firefly Pose,https://www.yogajournal.com/poses/firefly-pose/
3,Wrists,Four-Limbed Staff Pose | Chaturanga Dandasana,https://www.yogajournal.com/poses/four-limbed-...
4,Wrists,Handstand,https://www.yogajournal.com/poses/handstand/


In [16]:
set(yoga_pose_df[yoga_pose_df.anatomy=="Abs"]["pose_name"]).difference(set(yoga_pose_df[yoga_pose_df.anatomy=="Belly"]["pose_name"]))

{'Dancer Pose | Lord of the Dance Pose',
 'Extended Puppy Pose',
 'Forearm Plank | Dolphin Plank Pose',
 'King Pigeon Pose',
 'One-Legged King Pigeon Pose II',
 'Peacock Pose',
 'Plank Pose',
 'Pose Dedicated to the Sage Koundinya I',
 'Pose Dedicated to the Sage Koundinya II',
 'Pose Dedicated to the Sage Marichi III',
 'Rope Pose',
 'The Best Yoga Poses for Sacroiliac Joint Pain'}

In [17]:
yoga_pose_df = yoga_pose_df.groupby(['pose_name','url'])['anatomy'].apply(list).reset_index(name='Anatomies')

In [18]:
yoga_pose_df.head()

,pose_name,url,Anatomies
0,3 Ways to Modify Revolved Abdomen Pose,https://www.yogajournal.com/poses/types/twists...,[Sacrum]
1,Anatomy 101: Understanding Your Sacroiliac Joint,https://www.yogajournal.com/yoga-101/anatomy-1...,[Sacrum]
2,Bharadvaja’s Twist,https://www.yogajournal.com/poses/bharadvaja-s...,"[Back, Neck, Spine, Hips, Back, Shoulders, Bel..."
3,Big Toe Pose,https://www.yogajournal.com/poses/big-toe-pose/,"[Back, Thighs, Pelvis, Neck, Spine, Hips, Lung..."
4,Boat Pose,https://www.yogajournal.com/poses/full-boat-po...,"[Spine, Hips, Kidneys, Flexors, Belly, Abs]"


### Information Extracted

In [19]:
yoga_position = pd.read_csv("..\\Information Extraction\\Data Extracted\\Yoga_position_info_extracted.csv")

In [20]:
yoga_position.head()

,english_Name,position
0,Bharadvaja's Twist,['sit']
1,Big Toe Pose,['stand']
2,Boat Pose,['sit']
3,Bound Angle Pose,['sit']
4,Bow Pose,[]


In [21]:
yoga_position.shape

(92, 2)

## Meditation

### Meditation names

In [22]:
meditation_dict = {
                 'english_Name':[], 'sanskrit_Name':[], 'meditation_Type':[],'url': []}

with open('..\Data_Scrape_Yoga\yoga\Extracted\meditation_List.jsonl', 'r', encoding="utf8") as json_file:
    json_list = list(json_file)
    
for json_str in json_list:
    result = json.loads(json_str)
    
    meditation_dict['english_Name'].append(result['english_Name'])
    meditation_dict['sanskrit_Name'].append(result['sanskrit_Name'])
    meditation_dict['meditation_Type'].append(result['meditation_Type'])
    meditation_dict['url'].append(result['url'])

meditation_df = pd.DataFrame.from_dict(meditation_dict)    

In [23]:
meditation_df.head()

,english_Name,sanskrit_Name,meditation_Type,url
0,Big Mind Meditation,Dhyana,Meditation,https://www.yogajournal.com/poses/big-mind-med...
1,Breath Retention,Kumbhaka Pranayama,Pranayama Exercises & Poses,https://www.yogajournal.com/pranayama/breath-r...
2,Calm Heart Meditation,Dhyana,Meditation,https://www.yogajournal.com/poses/calm-heart-m...
3,Channel Cleaning Breath,Nadi Shodhana Pranayama,Pranayama Exercises & Poses,https://www.yogajournal.com/pranayama/channel-...
4,Conqueror Breath,Ujjayi Pranayama,Pranayama Exercises & Poses,https://www.yogajournal.com/pranayama/conquero...


### Meditation Details

In [24]:
meditation_details_dict = {
                 'name':[], 'instructions':[]}

with open('..\Data_Scrape_Yoga\yoga\Extracted\meditationdetails.jsonl', 'r', encoding="utf8") as json_file:
    json_list = list(json_file)
    
for json_str in json_list:
    result = json.loads(json_str)
    
    meditation_details_dict['name'].append(result['name'])
    meditation_details_dict['instructions'].append(result['instructions'])

meditation_detials_df = pd.DataFrame.from_dict(meditation_details_dict)    

In [25]:
meditation_detials_df.head()

,name,instructions
0,Upward Abdominal Lock,"[(oo-dee-YAH-nah BAHN-dah, uddiyana = upward (..."
1,Truth Seal,"[tattva, = reality, truth, mudra, = seal, Step..."
2,Wheel of Dharma Seal,"[Dharmachakra, = wheel of, Dharma, mudra, = se..."
3,Upright Seal,"[linga, = symbol, mudra, = seal, Step 1, Inter..."
4,Skull Shining Breath,"[(kah-pah-lah-BAH-tee), kapala = skull, bhati ..."


# Gym Details

In [26]:
exercise_dict = {
                 'id':[], 'name':[], 'muscle':[],'url': [], 'type':[], 'equipment':[], 'mechanics':[], 'difficulty':[]}

with open('..\\fitness_kg\\Extracted\\Gym\\exercise.jsonl', 'r', encoding="utf8") as json_file:
    json_list = list(json_file)
    
for json_str in json_list:
    result = json.loads(json_str)
#     print(result)
#     break
    exercise_dict['id'].append(result['id'])
    exercise_dict['name'].append(result['name'])    
    exercise_dict['url'].append(result['url'])
    exercise_dict['muscle'].append(result['muscle'])
    exercise_dict['type'].append(result['type'])
    exercise_dict['equipment'].append(result['equipment'])
    exercise_dict['mechanics'].append(result['mechanics'])
    exercise_dict['difficulty'].append(result['difficulty'])

exercise_df = pd.DataFrame.from_dict(exercise_dict)    

In [27]:
exercise_df.head()

,id,name,muscle,url,type,equipment,mechanics,difficulty
0,https://www.muscleandstrength.com/exercises/pl...,Plantar Fascia Lacrosse Ball,plantar-fascia,https://www.muscleandstrength.com/exercises/pl...,None,None,None,None
1,https://www.muscleandstrength.com/exercises/up...,Barbell Upright Row,traps,https://www.muscleandstrength.com/exercises/up...,strength,barbell,compound,beginner
2,https://www.muscleandstrength.com/exercises/du...,Dumbbell Shrug,traps,https://www.muscleandstrength.com/exercises/du...,strength,dumbbell,isolation,beginner
3,https://www.muscleandstrength.com/exercises/ba...,Barbell Shrug,traps,https://www.muscleandstrength.com/exercises/ba...,strength,barbell,isolation,beginner
4,https://www.muscleandstrength.com/exercises/wi...,Wide Grip Upright Row,traps,https://www.muscleandstrength.com/exercises/wi...,strength,barbell,compound,intermediate


In [28]:
exercise_df.muscle.unique()

array(['plantar-fascia', 'traps', 'palmar-fascia', 'triceps', 'neck.html',
       'shoulders', 'quads', 'obliques', 'it-band', 'hip-flexors',
       'middle-back', 'lower-back', 'hamstrings', 'glutes', 'lats',
       'forearms', 'chest', 'calves', 'biceps', 'adductors.html',
       'abductors.html', 'abs'], dtype=object)

### Exercise Group

In [289]:
exercise_group_dict = {
                 'id':[], 'name':[], 'url': [], 'muscle':[], 'type':[], 'equipment':[], 'mechanics':[], 
    'force_type': [], 'difficulty':[], 'overview': [], 'instructions':[]}

with open('..\\fitness_kg\\Extracted\\Gym\\exercise_group.jsonl', 'r', encoding="utf8") as json_file:
    json_list = list(json_file)
    
for json_str in json_list:
    result = json.loads(json_str)

    exercise_group_dict['id'].append(result['id'])
    exercise_group_dict['name'].append(result['name'])    
    exercise_group_dict['url'].append(result['url'])
    exercise_group_dict['muscle'].append(result['muscle'])
    exercise_group_dict['type'].append(result['type'])
    exercise_group_dict['equipment'].append(result['equipment'])
    exercise_group_dict['mechanics'].append(result['mechanics'])
    exercise_group_dict['force_type'].append(result['force_type'])
    exercise_group_dict['difficulty'].append(result['difficulty'])    
    exercise_group_dict['overview'].append(result['overview'])    
    exercise_group_dict['instructions'].append(result['instructions'])

exercise_group_df = pd.DataFrame.from_dict(exercise_group_dict)    

In [290]:
exercise_group_df.head() 

,id,name,url,muscle,type,equipment,mechanics,force_type,difficulty,overview,instructions
0,https://www.muscleandstrength.com/exercises/wi...,wide grip upright row,https://www.muscleandstrength.com/exercises/wi...,traps,strength,barbell,compound,pull,intermediate,,load up a barbell with the weight you want to ...
1,https://www.muscleandstrength.com/exercises/ba...,barbell shrug,https://www.muscleandstrength.com/exercises/ba...,traps,strength,barbell,isolation,pull (bilateral),beginner,the barbell shrug is a great exercise for thos...,position the safeties just below waist height ...
2,https://www.muscleandstrength.com/exercises/du...,dumbbell goblet squat,https://www.muscleandstrength.com/exercises/du...,quads,strength,dumbbell,compound,push (bilateral),beginner,the dumbbell goblet squat is a variation of th...,select a dumbbell and position it at chest hei...
3,https://www.muscleandstrength.com/exercises/du...,dumbbell rear lunge,https://www.muscleandstrength.com/exercises/du...,quads,strength,dumbbell,compound,push (unilateral),intermediate,the dumbbell reverse lunge is a dumbbell lunge...,set up with your feet shoulder width apart and...
4,https://www.muscleandstrength.com/exercises/du...,dumbbell split squat,https://www.muscleandstrength.com/exercises/du...,quads,strength,dumbbell,compound,push (unilateral),beginner,the dumbbell split squat is a dumbbell lunge v...,set up in a split stance position while grippi...


In [31]:
len(exercise_group_df[exercise_group_df.equipment == 'machine']["url"].values)

91

### Muscle Groups

In [32]:
muscle_group_dict = {
                 'id':[], 'name':[], 'url': []}

with open('..\\fitness_kg\\Extracted\\Gym\\muscle_groups.jsonl', 'r', encoding="utf8") as json_file:
    json_list = list(json_file)
    
for json_str in json_list:
    result = json.loads(json_str)
    

    muscle_group_dict['id'].append(result['id'])
    muscle_group_dict['name'].append(result['name'])    
    muscle_group_dict['url'].append(result['url'])

muscle_group_df = pd.DataFrame.from_dict(muscle_group_dict)    

In [33]:
muscle_group_df.head()

,id,name,url
0,https://www.muscleandstrength.com/exercises/ab...,Abductors,https://www.muscleandstrength.com/exercises/ab...
1,https://www.muscleandstrength.com/exercises/abs,Abs,https://www.muscleandstrength.com/exercises/abs
2,https://www.muscleandstrength.com/exercises/ad...,Adductors,https://www.muscleandstrength.com/exercises/ad...
3,https://www.muscleandstrength.com/exercises/bi...,Biceps,https://www.muscleandstrength.com/exercises/bi...
4,https://www.muscleandstrength.com/exercises/ca...,Calves,https://www.muscleandstrength.com/exercises/ca...


In [288]:
muscle_group_df.name.unique

<bound method Series.unique of 0          Abductors
1                Abs
2          Adductors
3             Biceps
4             Calves
5              Chest
6           Forearms
7             Glutes
8         Hamstrings
9        Hip Flexors
10           IT Band
11              Lats
12        Lower Back
13        Upper Back
14              Neck
15          Obliques
16     Palmar Fascia
17    Plantar Fascia
18             Quads
19         Shoulders
20             Traps
21           Triceps
Name: name, dtype: object>

### Information Extracted

In [34]:
exercise_additional_info = pd.read_csv("..\\Information Extraction\\Data Extracted\\Gym_info_extracted.csv")

In [35]:
exercise_additional_info.head()

,id,equipment,position
0,https://www.muscleandstrength.com/exercises/wi...,"['bar', 'barbell']",['stand']
1,https://www.muscleandstrength.com/exercises/ba...,['bar'],['stand']
2,https://www.muscleandstrength.com/exercises/du...,['dumbbell'],[]
3,https://www.muscleandstrength.com/exercises/du...,['dumbbell'],['stand']
4,https://www.muscleandstrength.com/exercises/du...,['dumbbell'],[]


# exRx

### ExrX Exercise

In [36]:
exrx_exercise_dict = {
                 'id':[], 'name':[], 'url': [], 'muscle':[], 'equipment':[]}

with open('..\\fitness_kg\\Extracted\\exrx\\exrx_exercise.jsonl', 'r', encoding="utf8") as json_file:
    json_list = list(json_file)
    
for json_str in json_list:
    result = json.loads(json_str)
#     print(result)
#     break
    exrx_exercise_dict['id'].append(result['id'])
    exrx_exercise_dict['name'].append(result['name'])    
    exrx_exercise_dict['url'].append(result['url'])
    exrx_exercise_dict['muscle'].append(result['muscle'])
    exrx_exercise_dict['equipment'].append(result['equipment'])

exrx_exercise_df = pd.DataFrame.from_dict(exrx_exercise_dict)

In [37]:
exrx_exercise_df.head()

,id,name,url,muscle,equipment
0,https://exrx.net/WeightExercises/Gastrocnemius...,safety calf raise,https://exrx.net/WeightExercises/Gastrocnemius...,https://exrx.net/Muscles/Gastrocnemius,Barbell\n
1,https://exrx.net/WeightExercises/Gastrocnemius...,standing calf raise,https://exrx.net/WeightExercises/Gastrocnemius...,https://exrx.net/Muscles/Gastrocnemius,Barbell\n
2,https://exrx.net/WeightExercises/Gastrocnemius...,standing calf raise,https://exrx.net/WeightExercises/Gastrocnemius...,https://exrx.net/Muscles/Gastrocnemius,Cable\n
3,https://exrx.net/WeightExercises/Gastrocnemius...,standing calf raise_single leg,https://exrx.net/WeightExercises/Gastrocnemius...,https://exrx.net/Muscles/Gastrocnemius,Cable\n
4,https://exrx.net/WeightExercises/Gastrocnemius...,standing calf raise_one arm,https://exrx.net/WeightExercises/Gastrocnemius...,https://exrx.net/Muscles/Gastrocnemius,Cable\n


### ExRx Exercise Group

In [38]:
exrx_exercise_group_dict = {
                 'id':[], 'name':[], 'url': [], 'mechanics':[], 'force_type':[], 'utility':[], 'instructions':[]}

with open('..\\fitness_kg\\Extracted\\exrx\\exrx_exercise_group.jsonl', 'r', encoding="utf8") as json_file:
    json_list = list(json_file)
    
for json_str in json_list:
    result = json.loads(json_str)
#     print(result)
#     break
    exrx_exercise_group_dict['id'].append(result['id'])
    exrx_exercise_group_dict['name'].append(result['name'])    
    exrx_exercise_group_dict['url'].append(result['url'])
    exrx_exercise_group_dict['mechanics'].append(result['mechanics'])
    exrx_exercise_group_dict['force_type'].append(result['force_type'])
    exrx_exercise_group_dict['utility'].append(result['utility'])
    exrx_exercise_group_dict['instructions'].append(result['instructions'])

exrx_exercise_group_df = pd.DataFrame.from_dict(exrx_exercise_group_dict)

In [39]:
exrx_exercise_group_df.head()

,id,name,url,mechanics,force_type,utility,instructions
0,https://exrx.net/WeightExercises/Gastrocnemius...,Dumbbell Standing Calf Raise,https://exrx.net/WeightExercises/Gastrocnemius...,isolated,push,[basic],Preparation:\ngrasp dumbbell in one hand to si...
1,https://exrx.net/WeightExercises/Gastrocnemius...,Lever 45° Calf Press (plate loaded),https://exrx.net/WeightExercises/Gastrocnemius...,isolated,push,[basic],Preparation:\nsit on seat with back on padded ...
2,https://exrx.net/WeightExercises/Gastrocnemius...,Lever Standing Calf Raise (plate loaded),https://exrx.net/WeightExercises/Gastrocnemius...,isolated,push,[basic],Preparation:\nplace shoulders under padded lev...
3,https://exrx.net/WeightExercises/Gastrocnemius...,Lever Calf Extension (plate loaded),https://exrx.net/WeightExercises/Gastrocnemius...,isolated,push,[basic],Preparation:\nsit on seat and position forefee...
4,https://exrx.net/WeightExercises/Gastrocnemius...,Lever Seated Calf Press (plate loaded),https://exrx.net/WeightExercises/Gastrocnemius...,isolated,push,[basic],Preparation:\nplace seat away from platform. s...


### exrx_muscle_groups

In [40]:
exrx_muscle_groups_dict = {
                 'id':[], 'name':[], 'url': [], 'parent_muscle':[]}

with open('..\\fitness_kg\\Extracted\\exrx\\exrx_muscle_groups.jsonl', 'r', encoding="utf8") as json_file:
    json_list = list(json_file)
    
for json_str in json_list:
    result = json.loads(json_str)
#     print(result)
#     break
    exrx_muscle_groups_dict['id'].append(result['id'])
    exrx_muscle_groups_dict['name'].append(result['name'])    
    exrx_muscle_groups_dict['url'].append(result['url'])
    exrx_muscle_groups_dict['parent_muscle'].append(result['parent_muscle'])

exrx_muscle_groups_df = pd.DataFrame.from_dict(exrx_muscle_groups_dict)

In [41]:
exrx_muscle_groups_df.head(12)

,id,name,url,parent_muscle
0,https://exrx.net/Lists/ExList/NeckWt,neck,https://exrx.net/Lists/ExList/NeckWt,None
1,https://exrx.net/Lists/ExList/ShouldWt,shoulders,https://exrx.net/Lists/ExList/ShouldWt,None
2,https://exrx.net/Lists/ExList/ArmWt,upper arms,https://exrx.net/Lists/ExList/ArmWt,None
3,https://exrx.net/Lists/ExList/ForeArmWt,forearms,https://exrx.net/Lists/ExList/ForeArmWt,None
4,https://exrx.net/Lists/ExList/BackWt,back,https://exrx.net/Lists/ExList/BackWt,None
5,https://exrx.net/Lists/ExList/ChestWt,chest,https://exrx.net/Lists/ExList/ChestWt,None
6,https://exrx.net/Lists/ExList/WaistWt,waist,https://exrx.net/Lists/ExList/WaistWt,None
7,https://exrx.net/Lists/ExList/HipsWt,hips,https://exrx.net/Lists/ExList/HipsWt,None
8,https://exrx.net/Lists/ExList/ThighWt,thighs,https://exrx.net/Lists/ExList/ThighWt,None
9,https://exrx.net/Lists/ExList/CalfWt,calves,https://exrx.net/Lists/ExList/CalfWt,None


### Link ExRx and gym exercise

In [42]:
df1 = exrx_muscle_groups_df[['id','name','parent_muscle']]
df2 = exrx_muscle_groups_df[['id','name']]

In [43]:
df_map = df1.merge(df2, right_on ='id',  left_on ='parent_muscle', how = 'left')[['name_x', 'name_y']].drop_duplicates()

In [44]:
df_map.to_csv("mapping.csv", index = False)

In [45]:
exrx_muscle_groups_df.name.unique()

array(['neck', 'shoulders', 'upper arms', 'forearms', 'back', 'chest',
       'waist', 'hips', 'thighs', 'calves', 'gastrocnemius', 'quadriceps',
       'brachioradialis', 'anterior deltoid', 'pectoralis major, sternal',
       'rectus abdominis', 'gluteus maximus', 'sternocleidomastoid',
       'triceps brachii', 'wrist flexors', 'splenius', 'soleus',
       'wrist extensors', 'tibialis anterior', 'pronators', 'supinators',
       'lateral deltoid', 'popliteus', 'posterior deltoid',
       'pectoralis major, clavicular', 'latissimus dorsi',
       'biceps brachii', 'transverse abdominis', 'obliques',
       'pectoralis minor', 'brachialis', 'serratus anterior',
       'supraspinatus', 'hamstrings', 'quadratus lumborum',
       'trapezius, upper fibers', 'erector spinae',
       'trapezius, middle fibers', 'trapezius, lower fibers', 'rhomboids',
       'infraspinatus', 'subscapularis', 'adductor magnus',
       'deep hip external rotators'], dtype=object)

In [46]:
muscle_group_df.name.unique()

array(['Abductors', 'Abs', 'Adductors', 'Biceps', 'Calves', 'Chest',
       'Forearms', 'Glutes', 'Hamstrings', 'Hip Flexors', 'IT Band',
       'Lats', 'Lower Back', 'Upper Back', 'Neck', 'Obliques',
       'Palmar Fascia', 'Plantar Fascia', 'Quads', 'Shoulders', 'Traps',
       'Triceps'], dtype=object)

# Fat

In [47]:
cardio_dict = {
                 'id':[], 'name':[], 'url': [], 'calories':[], 'met':[], 'intensity':[]}

with open('..\\fitness_kg\\Extracted\\Fat\\cardio.jsonl', 'r', encoding="utf8") as json_file:
    json_list = list(json_file)
    
for json_str in json_list:
    result = json.loads(json_str)
#     print(result)
#     break
    cardio_dict['id'].append(result['id'])
    cardio_dict['name'].append(result['name'])    
    cardio_dict['url'].append(result['url'])
    cardio_dict['calories'].append(result['calories'])
    cardio_dict['met'].append(result['met'])
    cardio_dict['intensity'].append(result['intensity'])

cardio_df = pd.DataFrame.from_dict(cardio_dict)

In [48]:
cardio_df.head()

,id,name,url,calories,met,intensity
0,https://www.fatsecret.com/fitness/sleeping,sleeping,https://www.fatsecret.com/fitness/sleeping,66,0.9,Light Intensity
1,https://www.fatsecret.com/fitness/resting,resting,https://www.fatsecret.com/fitness/resting,74,1.0,Light Intensity
2,https://www.fatsecret.com/fitness/sitting,sitting,https://www.fatsecret.com/fitness/sitting,103,1.4,Light Intensity
3,https://www.fatsecret.com/fitness/desk-work,desk work,https://www.fatsecret.com/fitness/desk-work,110,1.5,Light Intensity
4,https://www.fatsecret.com/fitness/driving,driving,https://www.fatsecret.com/fitness/driving,118,1.6,Light Intensity


# Add url for Anatomy and muscles in mapping file

In [49]:
### read the muscle sheet
muscle = pd.read_excel("mapping_master.xlsx", sheet_name = "Muscle") #Anatomy, 
muscle.head()

,NAME,URL
0,gastrocnemius,NaN
1,quadriceps,NaN
2,brachioradialis,NaN
3,anterior deltoid,NaN
4,"pectoralis major, sternal",NaN


In [50]:
exrx_url_dict = exrx_muscle_groups_df[['name','url']].set_index('name').to_dict()

muscle_group_req_df = muscle_group_df[['name','url']]
muscle_group_req_df['name'] = muscle_group_req_df['name'].str.lower()
mns_url_dict = muscle_group_req_df.set_index('name').to_dict()

yoga_categories_req_df = yoga_categories_df[['anatomy','url']]
yoga_categories_req_df['anatomy'] = yoga_categories_req_df['anatomy'].str.lower()
yoga_url_dict = yoga_categories_req_df.set_index('anatomy').to_dict()

C:\Users\shyam\AppData\Local\Temp/ipykernel_4780/4080422820.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  muscle_group_req_df['name'] = muscle_group_req_df['name'].str.lower()


In [51]:
def findURL(x):

    muscle_to_check = x["NAME"].lower()
    if muscle_to_check in exrx_url_dict['url']:
        return exrx_url_dict['url'][muscle_to_check]
    elif muscle_to_check in mns_url_dict['url']:
        return mns_url_dict['url'][muscle_to_check]
    else:
        return yoga_url_dict['url'][muscle_to_check]

In [52]:
muscle["URL"] = muscle.apply(lambda x: findURL(x), axis= 1)
muscle.isnull().sum()

NAME    0
URL     0
dtype: int64

In [295]:
muscle.head()

,NAME,URL
0,gastrocnemius,https://exrx.net/Muscles/Gastrocnemius
1,quadriceps,https://exrx.net/Muscles/Quadriceps
2,brachioradialis,https://exrx.net/Muscles/Brachioradialis
3,anterior deltoid,https://exrx.net/Muscles/DeltoidAnterior
4,"pectoralis major, sternal",https://exrx.net/Muscles/PectoralisSternal


In [297]:
muscle["NAME"] = muscle["NAME"].str.replace(", ", "-")

#### Add URL for anatomy sheet

In [53]:
anatomy = pd.read_excel("mapping_master.xlsx", sheet_name = "Anatomy") #, 
anatomy.head()

,NAME,URL
0,neck,NaN
1,shoulders,NaN
2,upper arms,NaN
3,forearms,NaN
4,back,NaN


In [54]:
anatomy["URL"] = anatomy.apply(lambda x: findURL(x), axis= 1)
anatomy.isnull().sum()

NAME    0
URL     0
dtype: int64

In [55]:
# anatomy.to_excel("mapping.xlsx", sheet_name = "Anatomy")

#### Write values to the mapped files

In [298]:
writer = pd.ExcelWriter('Mapping/mapped_final_.xlsx', engine='xlsxwriter')
anatomy.to_excel(writer,'Anatomy', index = False)
muscle.to_excel(writer,'Muscle', index = False)
writer.close()

# Adding group information (for blocking) to exrx and muscle & strength

#### Read the mapping file

In [57]:
grouping_file = pd.read_excel("mapping_master.xlsx", sheet_name = "Muscle&Anatomy")
grouping_file.fillna("", inplace = True)
grouping_file.head(10)

,Name,SAME_AS,PARENT,URL,Additional Url,Muscle|Anatomy
0,neck,,,,,Anatomy
1,shoulders,,,,,Anatomy
2,upper arms,,,,,Anatomy
3,forearms,,,,,Anatomy
4,back,,,,,Anatomy
5,chest,,,,,Anatomy
6,waist,,,,,Anatomy
7,hips,,,,,Anatomy
8,thighs,,,,,Anatomy
9,calves,,,,,Anatomy


#### Create url for all of the muscles and anatomy in exrx and muscle&strength

In [58]:
mapping_file_combined = pd.concat([muscle_group_df[['name','url']], exrx_muscle_groups_df[['name','url']]], axis = 0, ignore_index=True)

In [59]:
anatomy_url_dict = mapping_file_combined[['name','url']].set_index('name').to_dict()

In [60]:
anatomy_url_dict

{'url': {'Abductors': 'https://www.muscleandstrength.com/exercises/abductors.html',
  'Abs': 'https://www.muscleandstrength.com/exercises/abs',
  'Adductors': 'https://www.muscleandstrength.com/exercises/adductors.html',
  'Biceps': 'https://www.muscleandstrength.com/exercises/biceps',
  'Calves': 'https://www.muscleandstrength.com/exercises/calves',
  'Chest': 'https://www.muscleandstrength.com/exercises/chest',
  'Forearms': 'https://www.muscleandstrength.com/exercises/forearms',
  'Glutes': 'https://www.muscleandstrength.com/exercises/glutes',
  'Hamstrings': 'https://www.muscleandstrength.com/exercises/hamstrings',
  'Hip Flexors': 'https://www.muscleandstrength.com/exercises/hip-flexors',
  'IT Band': 'https://www.muscleandstrength.com/exercises/it-band',
  'Lats': 'https://www.muscleandstrength.com/exercises/lats',
  'Lower Back': 'https://www.muscleandstrength.com/exercises/lower-back',
  'Upper Back': 'https://www.muscleandstrength.com/exercises/middle-back',
  'Neck': 'https:/

In [61]:
def findURL(x, dictname):
    if len(x["PARENT"])>0:
        return dictname["url"][x["PARENT"]]
    elif len(x["SAME_AS"])>0:
        return dictname["url"][x["SAME_AS"]]
    else:
        return dictname["url"][x["Name"]]

In [62]:
grouping_file

,Name,SAME_AS,PARENT,URL,Additional Url,Muscle|Anatomy
0,neck,,,,,Anatomy
1,shoulders,,,,,Anatomy
2,upper arms,,,,,Anatomy
3,forearms,,,,,Anatomy
4,back,,,,,Anatomy
...,...,...,...,...,...,...
59,Glutes,,,,,muscle
60,Triceps,,,,,muscle
61,Lats,,,,,muscle
62,Biceps,,upper arms,,,muscle


#### Add the parent url to the grouping file

In [63]:
grouping_file["URL"] = grouping_file.apply(lambda x:  findURL(x, anatomy_url_dict), axis = 1)

In [64]:
grouping_file["Name"] = grouping_file["Name"].str.lower()

In [65]:
group_map_dict = grouping_file[["Name", "URL"]].set_index("Name").to_dict()

In [66]:
group_map_dict

{'URL': {'neck': 'https://exrx.net/Lists/ExList/NeckWt',
  'shoulders': 'https://exrx.net/Lists/ExList/ShouldWt',
  'upper arms': 'https://exrx.net/Lists/ExList/ArmWt',
  'forearms': 'https://exrx.net/Lists/ExList/ForeArmWt',
  'back': 'https://exrx.net/Lists/ExList/BackWt',
  'chest': 'https://exrx.net/Lists/ExList/ChestWt',
  'waist': 'https://exrx.net/Lists/ExList/WaistWt',
  'hips': 'https://exrx.net/Lists/ExList/HipsWt',
  'thighs': 'https://exrx.net/Lists/ExList/ThighWt',
  'calves': 'https://exrx.net/Lists/ExList/CalfWt',
  'gastrocnemius': 'https://exrx.net/Lists/ExList/CalfWt',
  'quadriceps': 'https://www.muscleandstrength.com/exercises/quads',
  'brachioradialis': 'https://exrx.net/Lists/ExList/ForeArmWt',
  'anterior deltoid': 'https://exrx.net/Lists/ExList/ChestWt',
  'pectoralis major, sternal': 'https://exrx.net/Lists/ExList/ChestWt',
  'rectus abdominis': 'https://www.muscleandstrength.com/exercises/abs',
  'gluteus maximus': 'https://www.muscleandstrength.com/exercises

# Add the updated url to the dataset

In [67]:
def findGroup(x, dictname, num):
    if num:
        muscle = x["muscle"].lower()
    else:
        muscle = x["name"].lower()
    if len(muscle)>0:
        return dictname["URL"][muscle].split("/")[-1]
    else:
        print("error: ", muscle)

### Add for the Muscle and Strength

In [68]:
exercise_group_df["group"] = exercise_group_df.apply(lambda x:  findGroup(x, group_map_dict, 1), axis = 1)

In [69]:
exercise_group_df.head(2)

,id,name,url,muscle,type,equipment,mechanics,force_type,difficulty,overview,instructions,group
0,https://www.muscleandstrength.com/exercises/wi...,wide grip upright row,https://www.muscleandstrength.com/exercises/wi...,traps,strength,barbell,compound,pull,intermediate,,load up a barbell with the weight you want to ...,traps
1,https://www.muscleandstrength.com/exercises/ba...,barbell shrug,https://www.muscleandstrength.com/exercises/ba...,traps,strength,barbell,isolation,pull (bilateral),beginner,the barbell shrug is a great exercise for thos...,position the safeties just below waist height ...,traps


### Add group for exrx exercises

In [70]:
exrx_muscle_groups_df["group"] = exrx_muscle_groups_df.apply(lambda x:  findGroup(x, group_map_dict, 0), axis = 1)

In [236]:
exrx_muscle_groups_df.head(15)

,id,name,url,parent_muscle,group
0,https://exrx.net/Lists/ExList/NeckWt,neck,https://exrx.net/Lists/ExList/NeckWt,None,NeckWt
1,https://exrx.net/Lists/ExList/ShouldWt,shoulders,https://exrx.net/Lists/ExList/ShouldWt,None,ShouldWt
2,https://exrx.net/Lists/ExList/ArmWt,upper arms,https://exrx.net/Lists/ExList/ArmWt,None,ArmWt
3,https://exrx.net/Lists/ExList/ForeArmWt,forearms,https://exrx.net/Lists/ExList/ForeArmWt,None,ForeArmWt
4,https://exrx.net/Lists/ExList/BackWt,back,https://exrx.net/Lists/ExList/BackWt,None,BackWt
5,https://exrx.net/Lists/ExList/ChestWt,chest,https://exrx.net/Lists/ExList/ChestWt,None,ChestWt
6,https://exrx.net/Lists/ExList/WaistWt,waist,https://exrx.net/Lists/ExList/WaistWt,None,WaistWt
7,https://exrx.net/Lists/ExList/HipsWt,hips,https://exrx.net/Lists/ExList/HipsWt,None,HipsWt
8,https://exrx.net/Lists/ExList/ThighWt,thighs,https://exrx.net/Lists/ExList/ThighWt,None,ThighWt
9,https://exrx.net/Lists/ExList/CalfWt,calves,https://exrx.net/Lists/ExList/CalfWt,None,CalfWt


In [72]:
exrx_exercise_df.head()

,id,name,url,muscle,equipment
0,https://exrx.net/WeightExercises/Gastrocnemius...,safety calf raise,https://exrx.net/WeightExercises/Gastrocnemius...,https://exrx.net/Muscles/Gastrocnemius,Barbell\n
1,https://exrx.net/WeightExercises/Gastrocnemius...,standing calf raise,https://exrx.net/WeightExercises/Gastrocnemius...,https://exrx.net/Muscles/Gastrocnemius,Barbell\n
2,https://exrx.net/WeightExercises/Gastrocnemius...,standing calf raise,https://exrx.net/WeightExercises/Gastrocnemius...,https://exrx.net/Muscles/Gastrocnemius,Cable\n
3,https://exrx.net/WeightExercises/Gastrocnemius...,standing calf raise_single leg,https://exrx.net/WeightExercises/Gastrocnemius...,https://exrx.net/Muscles/Gastrocnemius,Cable\n
4,https://exrx.net/WeightExercises/Gastrocnemius...,standing calf raise_one arm,https://exrx.net/WeightExercises/Gastrocnemius...,https://exrx.net/Muscles/Gastrocnemius,Cable\n


In [73]:
exrx_exercise_group_df.head()

,id,name,url,mechanics,force_type,utility,instructions
0,https://exrx.net/WeightExercises/Gastrocnemius...,Dumbbell Standing Calf Raise,https://exrx.net/WeightExercises/Gastrocnemius...,isolated,push,[basic],Preparation:\ngrasp dumbbell in one hand to si...
1,https://exrx.net/WeightExercises/Gastrocnemius...,Lever 45° Calf Press (plate loaded),https://exrx.net/WeightExercises/Gastrocnemius...,isolated,push,[basic],Preparation:\nsit on seat with back on padded ...
2,https://exrx.net/WeightExercises/Gastrocnemius...,Lever Standing Calf Raise (plate loaded),https://exrx.net/WeightExercises/Gastrocnemius...,isolated,push,[basic],Preparation:\nplace shoulders under padded lev...
3,https://exrx.net/WeightExercises/Gastrocnemius...,Lever Calf Extension (plate loaded),https://exrx.net/WeightExercises/Gastrocnemius...,isolated,push,[basic],Preparation:\nsit on seat and position forefee...
4,https://exrx.net/WeightExercises/Gastrocnemius...,Lever Seated Calf Press (plate loaded),https://exrx.net/WeightExercises/Gastrocnemius...,isolated,push,[basic],Preparation:\nplace seat away from platform. s...


In [74]:
## Added muscle information to the exrx_exercise_group file
exrx_exercise_group_df = exrx_exercise_group_df.merge(exrx_exercise_df[['url','muscle']], on="url", how = "left")
exrx_exercise_group_df.head()

,id,name,url,mechanics,force_type,utility,instructions,muscle
0,https://exrx.net/WeightExercises/Gastrocnemius...,Dumbbell Standing Calf Raise,https://exrx.net/WeightExercises/Gastrocnemius...,isolated,push,[basic],Preparation:\ngrasp dumbbell in one hand to si...,https://exrx.net/Muscles/Gastrocnemius
1,https://exrx.net/WeightExercises/Gastrocnemius...,Lever 45° Calf Press (plate loaded),https://exrx.net/WeightExercises/Gastrocnemius...,isolated,push,[basic],Preparation:\nsit on seat with back on padded ...,https://exrx.net/Muscles/Gastrocnemius
2,https://exrx.net/WeightExercises/Gastrocnemius...,Lever Standing Calf Raise (plate loaded),https://exrx.net/WeightExercises/Gastrocnemius...,isolated,push,[basic],Preparation:\nplace shoulders under padded lev...,https://exrx.net/Muscles/Gastrocnemius
3,https://exrx.net/WeightExercises/Gastrocnemius...,Lever Calf Extension (plate loaded),https://exrx.net/WeightExercises/Gastrocnemius...,isolated,push,[basic],Preparation:\nsit on seat and position forefee...,https://exrx.net/Muscles/Gastrocnemius
4,https://exrx.net/WeightExercises/Gastrocnemius...,Lever Seated Calf Press (plate loaded),https://exrx.net/WeightExercises/Gastrocnemius...,isolated,push,[basic],Preparation:\nplace seat away from platform. s...,https://exrx.net/Muscles/Gastrocnemius


In [75]:
exrx_muscle_groups_df["url"] = exrx_muscle_groups_df["url"].astype(str)
exrx_exercise_group_df["url"] = exrx_exercise_group_df["url"].astype(str)

In [76]:
## Add the group information to exrx exercise group
exrx_exercise_group_df = exrx_exercise_group_df.merge(exrx_muscle_groups_df[["url","group"]], left_on="muscle", right_on="url", how="left")
exrx_exercise_group_df.head()

,id,name,url_x,mechanics,force_type,utility,instructions,muscle,url_y,group
0,https://exrx.net/WeightExercises/Gastrocnemius...,Dumbbell Standing Calf Raise,https://exrx.net/WeightExercises/Gastrocnemius...,isolated,push,[basic],Preparation:\ngrasp dumbbell in one hand to si...,https://exrx.net/Muscles/Gastrocnemius,https://exrx.net/Muscles/Gastrocnemius,CalfWt
1,https://exrx.net/WeightExercises/Gastrocnemius...,Lever 45° Calf Press (plate loaded),https://exrx.net/WeightExercises/Gastrocnemius...,isolated,push,[basic],Preparation:\nsit on seat with back on padded ...,https://exrx.net/Muscles/Gastrocnemius,https://exrx.net/Muscles/Gastrocnemius,CalfWt
2,https://exrx.net/WeightExercises/Gastrocnemius...,Lever Standing Calf Raise (plate loaded),https://exrx.net/WeightExercises/Gastrocnemius...,isolated,push,[basic],Preparation:\nplace shoulders under padded lev...,https://exrx.net/Muscles/Gastrocnemius,https://exrx.net/Muscles/Gastrocnemius,CalfWt
3,https://exrx.net/WeightExercises/Gastrocnemius...,Lever Calf Extension (plate loaded),https://exrx.net/WeightExercises/Gastrocnemius...,isolated,push,[basic],Preparation:\nsit on seat and position forefee...,https://exrx.net/Muscles/Gastrocnemius,https://exrx.net/Muscles/Gastrocnemius,CalfWt
4,https://exrx.net/WeightExercises/Gastrocnemius...,Lever Seated Calf Press (plate loaded),https://exrx.net/WeightExercises/Gastrocnemius...,isolated,push,[basic],Preparation:\nplace seat away from platform. s...,https://exrx.net/Muscles/Gastrocnemius,https://exrx.net/Muscles/Gastrocnemius,CalfWt


In [77]:
#### Retain only the required colums
exrx_exercise_group_df.drop(columns = ['url_y'], inplace = True)
exrx_exercise_group_df.head()

,id,name,url_x,mechanics,force_type,utility,instructions,muscle,group
0,https://exrx.net/WeightExercises/Gastrocnemius...,Dumbbell Standing Calf Raise,https://exrx.net/WeightExercises/Gastrocnemius...,isolated,push,[basic],Preparation:\ngrasp dumbbell in one hand to si...,https://exrx.net/Muscles/Gastrocnemius,CalfWt
1,https://exrx.net/WeightExercises/Gastrocnemius...,Lever 45° Calf Press (plate loaded),https://exrx.net/WeightExercises/Gastrocnemius...,isolated,push,[basic],Preparation:\nsit on seat with back on padded ...,https://exrx.net/Muscles/Gastrocnemius,CalfWt
2,https://exrx.net/WeightExercises/Gastrocnemius...,Lever Standing Calf Raise (plate loaded),https://exrx.net/WeightExercises/Gastrocnemius...,isolated,push,[basic],Preparation:\nplace shoulders under padded lev...,https://exrx.net/Muscles/Gastrocnemius,CalfWt
3,https://exrx.net/WeightExercises/Gastrocnemius...,Lever Calf Extension (plate loaded),https://exrx.net/WeightExercises/Gastrocnemius...,isolated,push,[basic],Preparation:\nsit on seat and position forefee...,https://exrx.net/Muscles/Gastrocnemius,CalfWt
4,https://exrx.net/WeightExercises/Gastrocnemius...,Lever Seated Calf Press (plate loaded),https://exrx.net/WeightExercises/Gastrocnemius...,isolated,push,[basic],Preparation:\nplace seat away from platform. s...,https://exrx.net/Muscles/Gastrocnemius,CalfWt


In [78]:
print(exrx_exercise_group_df["muscle"].unique())

['https://exrx.net/Muscles/Gastrocnemius'
 'https://exrx.net/Muscles/Brachioradialis'
 'https://exrx.net/Muscles/Quadriceps'
 'https://exrx.net/Muscles/SerratusAnterior'
 'https://exrx.net/Muscles/Obliques'
 'https://exrx.net/Muscles/GluteusMaximus'
 'https://exrx.net/Muscles/TrapeziusUpper'
 'https://exrx.net/Muscles/HipExernalRotators'
 'https://exrx.net/Muscles/Hamstrings'
 'https://exrx.net/Lists/ExList/HipsWt'
 'https://exrx.net/Muscles/Adductors'
 'https://exrx.net/Lists/ExList/ThighWt'
 'https://exrx.net/Muscles/ErectorSpinae'
 'https://exrx.net/Muscles/Subscapularis'
 'https://exrx.net/Muscles/Infraspinatus'
 'https://exrx.net/Muscles/Rhomboids'
 'https://exrx.net/Muscles/LatissimusDorsi'
 'https://exrx.net/Muscles/Brachialis'
 'https://exrx.net/Muscles/DeltoidPosterior'
 'https://exrx.net/Muscles/BicepsBrachii'
 'https://exrx.net/Muscles/PectoralisClavicular'
 'https://exrx.net/Muscles/RectusAbdominis'
 'https://exrx.net/Muscles/TricepsBrachii'
 'https://exrx.net/Lists/ExList/

In [79]:
exrx_exercise_group_df.columns

Index(['id', 'name', 'url_x', 'mechanics', 'force_type', 'utility',
       'instructions', 'muscle', 'group'],
      dtype='object')

In [80]:
exrx_exercise_group_df[exrx_exercise_group_df.group.isnull()]

,id,name,url_x,mechanics,force_type,utility,instructions,muscle,group
870,https://exrx.net/WeightExercises/Biceps/BWUnde...,Inverted Biceps Row,https://exrx.net/WeightExercises/Biceps/BWUnde...,,,[],,NaN,NaN
871,https://exrx.net/WeightExercises/Biceps/BWUnde...,Inverted Biceps Row (high bar),https://exrx.net/WeightExercises/Biceps/BWUnde...,compound,pull,"[basic, auxiliary]",Preparation:\nstand facing arms length away fr...,NaN,NaN


In [81]:
exrx_exercise_group_df.dropna(inplace = True)

# Perform Blocking

In [82]:
import rltk
import csv
from math import comb
from difflib import SequenceMatcher
import random
# You can use this tokenizer in case you need to manipulate some data
tokenizer = rltk.tokenizer.crf_tokenizer.crf_tokenizer.CrfTokenizer()

In [83]:
#'id', 'name', 'url', 'muscle', 'type', 'equipment', 'mechanics','force_type', 'difficulty', 'overview', 'instructions', 'group'
class muscleNstrength(rltk.Record):
    def __init__(self, raw_object):
        super().__init__(raw_object)
        self.name = ''

    @rltk.cached_property
    def id(self):
        return self.raw_object['id']

    @rltk.cached_property
    def name_string(self):
        return self.raw_object['name']

    @rltk.cached_property
    def name_tokens(self):
        return set(tokenizer.tokenize(self.name_string))
    
    @rltk.cached_property
    def muscle(self):
        return self.raw_object['muscle']
    
    @rltk.cached_property
    def group(self):
        return self.raw_object['group']

#'id', 'name', 'url_x', 'mechanics', 'force_type', 'utility', 'instructions', 'muscle', 'group'
class exrx(rltk.Record):
    def __init__(self, raw_object):
        super().__init__(raw_object)
        self.name = ''

    @rltk.cached_property
    def id(self):
        return self.raw_object['id']

    @rltk.cached_property
    def name_string(self):
        return self.raw_object['name']
    
    @rltk.cached_property
    def name_tokens(self):
        return set(tokenizer.tokenize(self.name_string))
    
    @rltk.cached_property
    def muscle(self):
        return self.raw_object['muscle'].split("/")[-1]
    
    @rltk.cached_property
    def group(self):
        return self.raw_object['group']
    
    @rltk.cached_property
    def force_type(self):
        return self.raw_object['force_type']


In [84]:
ds1 = rltk.Dataset(rltk.DataFrameReader(exercise_group_df), record_class=muscleNstrength)
ds2 = rltk.Dataset(rltk.DataFrameReader(exrx_exercise_group_df), record_class=exrx)

In [85]:
# print some entries
print(ds1.generate_dataframe().shape)
print(ds2.generate_dataframe().shape)

print(ds1.generate_dataframe().head(5))
print(ds2.generate_dataframe().head(15))




(1076, 5)
(1611, 6)
                                                  id            name_string  \
0  https://www.muscleandstrength.com/exercises/wi...  wide grip upright row   
1  https://www.muscleandstrength.com/exercises/ba...          barbell shrug   
2  https://www.muscleandstrength.com/exercises/du...  dumbbell goblet squat   
3  https://www.muscleandstrength.com/exercises/du...    dumbbell rear lunge   
4  https://www.muscleandstrength.com/exercises/du...   dumbbell split squat   

                  name_tokens muscle  group  
0  {wide, upright, grip, row}  traps  traps  
1            {barbell, shrug}  traps  traps  
2   {goblet, dumbbell, squat}  quads  quads  
3     {lunge, rear, dumbbell}  quads  quads  
4    {split, squat, dumbbell}  quads  quads  
                                                   id  \
0   https://exrx.net/WeightExercises/Gastrocnemius...   
1   https://exrx.net/WeightExercises/Gastrocnemius...   
2   https://exrx.net/WeightExercises/Gastrocnemius...   
3

In [86]:
## Blocking on group
bg = rltk.HashBlockGenerator()

def blocking(x):
    if type(x.group) != 'str':
        print(x.group)
    return x.group

block = bg.generate(
    bg.block(ds1, function_=lambda x: x.group),
    bg.block(ds2, function_=lambda x: x.group)
)

In [87]:
print("Total comparisons before blocking:= ", ds1.generate_dataframe().shape[0]*ds2.generate_dataframe().shape[0])

Total comparisons before blocking:=  1733436


# Entity Linking

In [88]:
def name_similarity(r1, r2):
    s1 = r1.name_string.lower()
    s1 = "".join(sorted(s1))
    s2 = r2.name_string.lower()
    s2 = "".join(sorted(s2))
    
    return rltk.levenshtein_similarity(s1,s2)

def token_similarity(r1, r2):
    s1 = list(r1.name_tokens)
    s2 = list(r2.name_tokens)
    return rltk.monge_elkan_similarity(s1,s2)


MY_TRESH = 0.85
def rule_based_method(r1, r2):
    score_1 = name_similarity(r1, r2)
    score_2 = token_similarity(r1, r2)
    
    total = 0.5*score_1 + 0.5*score_2
    
    return score_2 > MY_TRESH, total

In [89]:
s1 = "rope tricep extension"
s2 = "Suspended Triceps Extension (with rope)"
rltk.levenshtein_similarity("".join(sorted(s1)), "".join(sorted(s2)))

0.5384615384615384

In [90]:
rltk.monge_elkan_similarity(tokenizer.tokenize("rope tricep extension"),
                                      tokenizer.tokenize("Suspended Triceps Extension (with rope)"))

0.9841269841269842

In [91]:
output_df = {"muscleNstrength":[], "exRx":[], "result":[], "confidence":[], "exrx_muscle": []}

from time import time
start = time()
pairs = rltk.get_record_pairs(ds1, ds2, block=block)
for r1, r2 in pairs:
    result, confidence = rule_based_method(r1, r2)
    
    if result:
        output_df['muscleNstrength'].append(r1.name_string)
        output_df['exRx'].append(r2.name_string)
        output_df['result'].append(result)
        output_df['confidence'].append(confidence)
        
        output_df['exrx_muscle'].append(r2.muscle)

In [92]:
count=0
pairs = rltk.get_record_pairs(ds1, ds2, block=block)
for r1, r2 in pairs:
    count+=1
    print(r1.name_string , r2.name_string)
    if count > 5:
        break
count

seated dumbbell shrug Cable Bar Shrug
seated dumbbell shrug Trap Bar Shrug
seated dumbbell shrug Chair Trapezius Stretch
seated dumbbell shrug Lever Narrow Grip Shrug (plate loaded)
seated dumbbell shrug Suspended Inverted Shrug
seated dumbbell shrug Cable Shrug (dual pulley)


6

In [93]:
block_matched = pd.DataFrame.from_dict(output_df)

In [94]:
block_matched.result.value_counts()

True    3059
Name: result, dtype: int64

In [95]:
muscleNStrength_map_submuscle = block_matched[["muscleNstrength", "exrx_muscle"]].drop_duplicates()

In [97]:
block_matched[["muscleNstrength", "exRx", "confidence"]].to_csv("BlockedNmatched_pairs.csv", index = False)

#### Add submuscle info 

In [98]:
block_matched[["exRx"]].drop_duplicates().merge(exrx_exercise_group_df[["name", "muscle"]], left_on="exRx", right_on="name",how="left")

,exRx,name,muscle
0,Cambered Barbell Seated Shrug,Cambered Barbell Seated Shrug,https://exrx.net/Muscles/TrapeziusUpper
1,Cable Bar Shrug,Cable Bar Shrug,https://exrx.net/Muscles/TrapeziusUpper
2,Cable Shrug (dual pulley),Cable Shrug (dual pulley),https://exrx.net/Muscles/TrapeziusUpper
3,Lever Seated Shrug (plate loaded),Lever Seated Shrug (plate loaded),https://exrx.net/Muscles/TrapeziusUpper
4,Cable Shrug (with stirrups),Cable Shrug (with stirrups),https://exrx.net/Muscles/TrapeziusUpper
...,...,...,...
951,"Lever Triceps Extension (no seat, plate loaded)","Lever Triceps Extension (no seat, plate loaded)",https://exrx.net/Muscles/TricepsBrachii
952,Dumbbell Triceps Extension,Dumbbell Triceps Extension,https://exrx.net/Muscles/TricepsBrachii
953,Dumbbell One Arm Triceps Extension,Dumbbell One Arm Triceps Extension,https://exrx.net/Muscles/TricepsBrachii
954,Bench Dip,Bench Dip,https://exrx.net/Muscles/TricepsBrachii


### Find out any missing exercises that did not get blocked

In [99]:
exrx_exercises = set(exrx_exercise_group_df.name)
exrx_blocked = set()
exrx_missing = set()

musclenstren = set(exercise_group_df.name)
musclenstren_blocked = set()
musclenstren_missing = set()

pairs = rltk.get_record_pairs(ds1, ds2, block=block)
for r1,r2 in pairs:
    musclenstren_blocked.add(r1.name_string)
    exrx_blocked.add(r2.name_string)
        
exrx_missing = exrx_exercises.difference(exrx_blocked)
musclenstren_missing = musclenstren.difference(musclenstren_blocked)

In [100]:
len(exrx_missing)

0

In [101]:
len(musclenstren_missing)

2

In [102]:
exercise_group_df[exercise_group_df.name.isin(list(musclenstren_missing))]

,id,name,url,muscle,type,equipment,mechanics,force_type,difficulty,overview,instructions,group
1050,https://www.muscleandstrength.com/exercises/pa...,palmar fascia lacrosse ball,https://www.muscleandstrength.com/exercises/pa...,palmar fascia,smr,lacrosse ball,isolation,compression,intermediate,using a lacrosse ball to perform smr on your p...,place the lacrosse ball directly underneath th...,palmar-fascia
1073,https://www.muscleandstrength.com/exercises/pl...,plantar fascia lacrosse ball,https://www.muscleandstrength.com/exercises/pl...,plantar fascia,smr,lacrosse ball,isolation,compression,intermediate,using a lacrosse ball to perform smr on your p...,"in a standing position, place the lacrosse bal...",plantar-fascia


In [103]:
exercise_group_df[exercise_group_df.name.isin(list(musclenstren_missing))]

,id,name,url,muscle,type,equipment,mechanics,force_type,difficulty,overview,instructions,group
1050,https://www.muscleandstrength.com/exercises/pa...,palmar fascia lacrosse ball,https://www.muscleandstrength.com/exercises/pa...,palmar fascia,smr,lacrosse ball,isolation,compression,intermediate,using a lacrosse ball to perform smr on your p...,place the lacrosse ball directly underneath th...,palmar-fascia
1073,https://www.muscleandstrength.com/exercises/pl...,plantar fascia lacrosse ball,https://www.muscleandstrength.com/exercises/pl...,plantar fascia,smr,lacrosse ball,isolation,compression,intermediate,using a lacrosse ball to perform smr on your p...,"in a standing position, place the lacrosse bal...",plantar-fascia


In [104]:
exercise_group_df.to_excel("test_exercise_group_df.xlsx", index = False)

In [105]:
exrx_exercise_group_df.to_excel("test_exrx_exercise_group_df.xlsx", index = False)

### In the mapped exercises find the ones that do not have a match

In [106]:
list(musclenstren.difference(set(block_matched["muscleNstrength"])))[:15]

['self-assisted straight bar dip',
 'one-leg standing bodyweight calf raise',
 'smith machine shoulder press behind neck',
 'standing banded face pull',
 'banded skullcrusher',
 'floor barbell twist',
 'alternating dumbbell bench press on exercise ball',
 'one-arm dumbbell upright row',
 'lying dumbbell front raise on incline bench',
 '90/90 hip crossover',
 'neutral grip decline dumbbell bench press',
 'z press',
 'exercise ball dead bug',
 'one arm neutral dumbbell wrist curl over bench',
 'alternate bent-over dumbbell kickback']

In [107]:
len((musclenstren.difference(set(block_matched["muscleNstrength"]))))

648

In [108]:
block_matched.groupby("muscleNstrength").count()["exRx"].value_counts()

1     83
2     62
3     52
4     48
5     25
8     22
7     17
6     15
10    12
13    10
14    10
12    10
9      8
11     5
24     4
16     4
20     3
21     3
17     3
39     3
23     3
19     3
28     2
22     2
33     2
32     2
30     2
31     2
65     1
35     1
42     1
27     1
49     1
34     1
36     1
29     1
40     1
56     1
15     1
Name: exRx, dtype: int64

In [109]:
## WIthin same muscle group and within equipmenyt whether they are close enough

# # Create the KG

### Muscle Strength

In [247]:
matched_mNs = block_matched[["muscleNstrength", "exRx"]].drop_duplicates().merge(exrx_exercise_group_df[["name", "muscle"]], left_on="exRx", right_on="name",how="left")
matched_mNs.head()

,muscleNstrength,exRx,name,muscle
0,seated dumbbell shrug,Cambered Barbell Seated Shrug,Cambered Barbell Seated Shrug,https://exrx.net/Muscles/TrapeziusUpper
1,seated cable shrug,Cable Bar Shrug,Cable Bar Shrug,https://exrx.net/Muscles/TrapeziusUpper
2,seated cable shrug,Cable Shrug (dual pulley),Cable Shrug (dual pulley),https://exrx.net/Muscles/TrapeziusUpper
3,seated cable shrug,Lever Seated Shrug (plate loaded),Lever Seated Shrug (plate loaded),https://exrx.net/Muscles/TrapeziusUpper
4,seated cable shrug,Cable Shrug (with stirrups),Cable Shrug (with stirrups),https://exrx.net/Muscles/TrapeziusUpper


In [248]:
###Get the original muscle name from exrx muscle groups
matched_mNs = matched_mNs.merge(exrx_muscle_groups_df[["name","url"]], left_on="muscle", right_on="url").\
drop(columns=["url","name_x","muscle"]).rename(columns={"name_y":"muscle"})
matched_mNs.head()

,muscleNstrength,exRx,muscle
0,seated dumbbell shrug,Cambered Barbell Seated Shrug,"trapezius, upper fibers"
1,seated cable shrug,Cable Bar Shrug,"trapezius, upper fibers"
2,seated cable shrug,Cable Shrug (dual pulley),"trapezius, upper fibers"
3,seated cable shrug,Lever Seated Shrug (plate loaded),"trapezius, upper fibers"
4,seated cable shrug,Cable Shrug (with stirrups),"trapezius, upper fibers"


In [294]:
matched_mNs["muscle"] = matched_mNs["muscle"].str.replace(", ","-")
matched_mNs.head()

,muscleNstrength,exRx,muscle
0,seated dumbbell shrug,Cambered Barbell Seated Shrug,trapezius-upper fibers
1,seated cable shrug,Cable Bar Shrug,trapezius-upper fibers
2,seated cable shrug,Cable Shrug (dual pulley),trapezius-upper fibers
3,seated cable shrug,Lever Seated Shrug (plate loaded),trapezius-upper fibers
4,seated cable shrug,Cable Shrug (with stirrups),trapezius-upper fibers


In [299]:
## For the matched MnS exercises get the muscle and force_type from exrxy
mns_additional_info = matched_mNs.groupby(['muscleNstrength']).agg({'muscle':lambda x: list(set(x))}).reset_index() #, 'force_type': lambda x: list(set(x))

In [300]:
mns_additional_info.head()

,muscleNstrength,muscle
0,45 degree leg press calf raise,"[tibialis anterior, gastrocnemius]"
1,ab crunch,"[rectus abdominis, obliques]"
2,alternate dumbbell lateral raise,"[posterior deltoid, lateral deltoid]"
3,alternate dumbbell preacher curl,[brachialis]
4,alternate incline dumbbell curl,"[biceps brachii, brachialis]"


In [301]:
exercise_group_df_merged = exercise_group_df.merge(mns_additional_info, left_on="name", right_on="muscleNstrength", how = "left").drop(columns="muscleNstrength")

In [302]:
exercise_group_df_merged.shape

(1076, 12)

In [303]:
exercise_group_df_merged.head()

,id,name,url,muscle_x,type,equipment,mechanics,force_type,difficulty,overview,instructions,muscle_y
0,https://www.muscleandstrength.com/exercises/wi...,wide grip upright row,https://www.muscleandstrength.com/exercises/wi...,traps,strength,barbell,compound,pull,intermediate,,load up a barbell with the weight you want to ...,NaN
1,https://www.muscleandstrength.com/exercises/ba...,barbell shrug,https://www.muscleandstrength.com/exercises/ba...,traps,strength,barbell,isolation,pull (bilateral),beginner,the barbell shrug is a great exercise for thos...,position the safeties just below waist height ...,[trapezius-upper fibers]
2,https://www.muscleandstrength.com/exercises/du...,dumbbell goblet squat,https://www.muscleandstrength.com/exercises/du...,quads,strength,dumbbell,compound,push (bilateral),beginner,the dumbbell goblet squat is a variation of th...,select a dumbbell and position it at chest hei...,"[quadriceps, gluteus maximus]"
3,https://www.muscleandstrength.com/exercises/du...,dumbbell rear lunge,https://www.muscleandstrength.com/exercises/du...,quads,strength,dumbbell,compound,push (unilateral),intermediate,the dumbbell reverse lunge is a dumbbell lunge...,set up with your feet shoulder width apart and...,"[quadriceps, gluteus maximus]"
4,https://www.muscleandstrength.com/exercises/du...,dumbbell split squat,https://www.muscleandstrength.com/exercises/du...,quads,strength,dumbbell,compound,push (unilateral),beginner,the dumbbell split squat is a dumbbell lunge v...,set up in a split stance position while grippi...,"[quadriceps, gluteus maximus]"


In [304]:
exercise_group_df_merged.fillna("", inplace= True)

In [305]:
exercise_group_df_merged["muscle"] = exercise_group_df_merged[["muscle_x", "muscle_y"]].apply(lambda x: x[1] + [x[0]] if (len(x[1])>0) else [x[0]], axis = 1)

In [306]:
exercise_group_df_merged.head()

,id,name,url,muscle_x,type,equipment,mechanics,force_type,difficulty,overview,instructions,muscle_y,muscle
0,https://www.muscleandstrength.com/exercises/wi...,wide grip upright row,https://www.muscleandstrength.com/exercises/wi...,traps,strength,barbell,compound,pull,intermediate,,load up a barbell with the weight you want to ...,,[traps]
1,https://www.muscleandstrength.com/exercises/ba...,barbell shrug,https://www.muscleandstrength.com/exercises/ba...,traps,strength,barbell,isolation,pull (bilateral),beginner,the barbell shrug is a great exercise for thos...,position the safeties just below waist height ...,[trapezius-upper fibers],"[trapezius-upper fibers, traps]"
2,https://www.muscleandstrength.com/exercises/du...,dumbbell goblet squat,https://www.muscleandstrength.com/exercises/du...,quads,strength,dumbbell,compound,push (bilateral),beginner,the dumbbell goblet squat is a variation of th...,select a dumbbell and position it at chest hei...,"[quadriceps, gluteus maximus]","[quadriceps, gluteus maximus, quads]"
3,https://www.muscleandstrength.com/exercises/du...,dumbbell rear lunge,https://www.muscleandstrength.com/exercises/du...,quads,strength,dumbbell,compound,push (unilateral),intermediate,the dumbbell reverse lunge is a dumbbell lunge...,set up with your feet shoulder width apart and...,"[quadriceps, gluteus maximus]","[quadriceps, gluteus maximus, quads]"
4,https://www.muscleandstrength.com/exercises/du...,dumbbell split squat,https://www.muscleandstrength.com/exercises/du...,quads,strength,dumbbell,compound,push (unilateral),beginner,the dumbbell split squat is a dumbbell lunge v...,set up in a split stance position while grippi...,"[quadriceps, gluteus maximus]","[quadriceps, gluteus maximus, quads]"


In [308]:
exercise_group_df_merged.drop(columns = ["muscle_x", "muscle_y"], inplace = True)

In [309]:
exercise_group_df_merged.head()

,id,name,url,type,equipment,mechanics,force_type,difficulty,overview,instructions,muscle
0,https://www.muscleandstrength.com/exercises/wi...,wide grip upright row,https://www.muscleandstrength.com/exercises/wi...,strength,barbell,compound,pull,intermediate,,load up a barbell with the weight you want to ...,[traps]
1,https://www.muscleandstrength.com/exercises/ba...,barbell shrug,https://www.muscleandstrength.com/exercises/ba...,strength,barbell,isolation,pull (bilateral),beginner,the barbell shrug is a great exercise for thos...,position the safeties just below waist height ...,"[trapezius-upper fibers, traps]"
2,https://www.muscleandstrength.com/exercises/du...,dumbbell goblet squat,https://www.muscleandstrength.com/exercises/du...,strength,dumbbell,compound,push (bilateral),beginner,the dumbbell goblet squat is a variation of th...,select a dumbbell and position it at chest hei...,"[quadriceps, gluteus maximus, quads]"
3,https://www.muscleandstrength.com/exercises/du...,dumbbell rear lunge,https://www.muscleandstrength.com/exercises/du...,strength,dumbbell,compound,push (unilateral),intermediate,the dumbbell reverse lunge is a dumbbell lunge...,set up with your feet shoulder width apart and...,"[quadriceps, gluteus maximus, quads]"
4,https://www.muscleandstrength.com/exercises/du...,dumbbell split squat,https://www.muscleandstrength.com/exercises/du...,strength,dumbbell,compound,push (unilateral),beginner,the dumbbell split squat is a dumbbell lunge v...,set up in a split stance position while grippi...,"[quadriceps, gluteus maximus, quads]"


#### Read the SAME_AS file

In [320]:
same_as_df = pd.read_excel("Mapping/mapping_master.xlsx", sheet_name="SAME_AS").dropna()

In [321]:
same_as_map = same_as_df.set_index("Name").to_dict()
same_as_map

{'SAME_AS': {'quadriceps': 'Quads',
  'gluteus maximus': 'Glutes',
  'triceps brachii': 'Triceps',
  'latissimus dorsi': 'Lats',
  'biceps brachii': 'Biceps'}}

In [311]:
def replace_muscle(muscle_list):
    return_list = []
    i=0
    for muscle in muscle_list:
        if muscle in same_as_map["SAME_AS"]:
#             print(muscle, same_as_map["SAME_AS"][muscle])
            return_list.append(same_as_map["SAME_AS"][muscle].lower())
        else:
            return_list.append(muscle.lower())
#             print(muscle)            
    return list(set(return_list))

In [314]:
exercise_group_df_merged["muscle"][:5]

0                            [traps]
1    [trapezius-upper fibers, traps]
2                    [quads, glutes]
3                    [quads, glutes]
4                    [quads, glutes]
Name: muscle, dtype: object

In [409]:
exercise_group_df_merged.difficulty.unique()

array(['intermediate', 'beginner', 'advanced'], dtype=object)

In [ ]:
forcetype-> pull

In [313]:
exercise_group_df_merged["muscle"] = exercise_group_df_merged["muscle"].apply(lambda x: replace_muscle(x))

In [328]:
exercise_group_df_merged.to_csv("Data_KG Creation/exercise.csv", index = False)

## Muscles

In [318]:
muscles = pd.read_excel("Mapping/mapped_final_.xlsx", sheet_name = "Muscle") #Anatomy
muscles.head()

,NAME,URL
0,gastrocnemius,https://exrx.net/Muscles/Gastrocnemius
1,quadriceps,https://exrx.net/Muscles/Quadriceps
2,brachioradialis,https://exrx.net/Muscles/Brachioradialis
3,anterior deltoid,https://exrx.net/Muscles/DeltoidAnterior
4,pectoralis major-sternal,https://exrx.net/Muscles/PectoralisSternal


In [324]:
muscles["SAME_AS"] = muscles["NAME"].map(same_as_map["SAME_AS"])
muscles.head()

,NAME,URL,SAME_AS
0,gastrocnemius,https://exrx.net/Muscles/Gastrocnemius,NaN
1,quadriceps,https://exrx.net/Muscles/Quadriceps,Quads
2,brachioradialis,https://exrx.net/Muscles/Brachioradialis,NaN
3,anterior deltoid,https://exrx.net/Muscles/DeltoidAnterior,NaN
4,pectoralis major-sternal,https://exrx.net/Muscles/PectoralisSternal,NaN


In [325]:
muscles["NAME"] = muscles["NAME"].str.lower()

In [327]:
muscles["SAME_AS"] = muscles["SAME_AS"].str.lower()

In [330]:
muscles.head()

,NAME,URL,SAME_AS
0,gastrocnemius,https://exrx.net/Muscles/Gastrocnemius,NaN
1,quadriceps,https://exrx.net/Muscles/Quadriceps,quads
2,brachioradialis,https://exrx.net/Muscles/Brachioradialis,NaN
3,anterior deltoid,https://exrx.net/Muscles/DeltoidAnterior,NaN
4,pectoralis major-sternal,https://exrx.net/Muscles/PectoralisSternal,NaN


In [329]:
muscles.to_csv("Data_KG Creation/muscles.csv", index = False)

### Anatomy

In [335]:
anatomy = pd.read_excel("Mapping/mapped_final_.xlsx", sheet_name = "Anatomy") #
anatomy.head()

,NAME,URL
0,neck,https://exrx.net/Lists/ExList/NeckWt
1,shoulders,https://exrx.net/Lists/ExList/ShouldWt
2,upper arms,https://exrx.net/Lists/ExList/ArmWt
3,forearms,https://exrx.net/Lists/ExList/ForeArmWt
4,back,https://exrx.net/Lists/ExList/BackWt


In [337]:
anatomy["NAME"] = anatomy["NAME"].str.lower()

In [338]:
anatomy.to_csv("Data_KG Creation/anatomy.csv", index = False)

### Equipment

In [344]:
equipment_df  = exercise_group_df[["name", "equipment"]]

In [349]:
equipment_df.head(10)

,name,equipment
0,wide grip upright row,barbell
1,barbell shrug,barbell
2,dumbbell goblet squat,dumbbell
3,dumbbell rear lunge,dumbbell
4,dumbbell split squat,dumbbell
5,dumbbell step up,dumbbell
6,reverse bodyweight lunge,bodyweight
7,smith machine squat,machine
8,barbell hack squat,barbell
9,superman,bodyweight


In [352]:
"smith machine squat".split("machine")

['smith ', ' squat']

In [373]:
def replaceEquipments(x):
    if "machine" in x[1].lower():
        if "machine" in x[0].lower():
#             print(x[0].lower().split("machine")[0] + "machine| ", x[0])
            return x[0].lower().split("machine")[0] + "machine"
        else:
#             print(x[0].lower() + " machine| ", x[0])
            return x[0].lower() + " machine"
    else:
        return x[1].lower()

In [430]:
equipments = equipment_df.apply(lambda x:  replaceEquipments(x) , axis = 1)
ex_mch_lst = ["leg press", "toe raise", "lat pull down", "smith", "back extension", "hack squat", 
              "cable chop", "bench press", "leg curl"]

In [431]:
equipment_set = set(equipments)

In [432]:
len(equipment_set)

82

### Read the extracted information

In [383]:
mns_info_extracted = pd.read_csv("../Information Extraction/Data Extracted/Gym_info_extracted.csv")

In [427]:
for equipments in mns_info_extracted["equipment"].values:
    for equipment in eval(equipments):
#         print(equipment)
        if len(equipment.strip())>0:
            equipment_set.add(equipment.strip())

In [429]:
len(equipment_set)

88

In [437]:
# equip_writer = csv.writer(open("Data_KG Creation/equipment.csv",'w'))

with open('Data_KG Creation/equipment.csv', 'w') as notables:
    file_write=csv.writer(notables, delimiter = '\n', quotechar='"', quoting=csv.QUOTE_MINIMAL)

    # write
    # TO DO: how to get the values to write as well?
    file_write.writerow("Equipment")
    file_write.writerow(list(equipment_set))

## Meditation details